In [1]:
!pip install pyspark


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from pyspark.sql import SparkSession , SQLContext
from pyspark import SparkContext

In [6]:
from pyspark import SparkContext

try:
    sc = SparkContext.getOrCreate()
    sc.stop()
    print("Stopped old SparkContext")
except Exception as e:
    print("No existing SparkContext:", e)


No existing SparkContext: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: org.apache.spark.SparkException: Invalid Spark URL: spark://HeartbeatReceiver@Ahmed_shady:64580
	at org.apache.spark.rpc.RpcEndpointAddress$.apply(RpcEndpointAddress.scala:66)
	at org.apache.spark.rpc.netty.NettyRpcEnv.asyncSetupEndpointRefByURI(NettyRpcEnv.scala:140)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.executor.Executor.<init>(Executor.scala:250)
	at org.apache.spark.scheduler.local.LocalEndpoint.<init>(LocalSchedulerBackend.scala:64)
	at org.apache.spark.scheduler.local.LocalSchedulerBackend.start(LocalSchedulerBackend.scala:132)
	at org.apache.spark.scheduler.TaskSchedulerImpl.start(TaskSchedulerImpl.scala:235)
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:590)
	at org.apa

In [5]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("MongoConnection") \
    .master("local[*]") \
    .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1") \
    .config("spark.mongodb.input.uri", "mongodb://localhost:27017/car_system.detections") \
    .config("spark.mongodb.output.uri", "mongodb://localhost:27017/car_system.detections") \
    .getOrCreate()

print("Spark Started!!!")

df = spark.read.format("com.mongodb.spark.sql.DefaultSource").load()
df.show()


Spark Started!!!
+--------------------+--------------------+-------------------+
|                 _id|          detections|          timestamp|
+--------------------+--------------------+-------------------+
|{6901ed73e541daec...|[{1, person, 0.74...|2025-10-29 13:33:22|
|{6901ed73e541daec...|[{1, person, 0.77...|2025-10-29 13:33:23|
|{6901ed73e541daec...|[{1, person, 0.79...|2025-10-29 13:33:23|
|{6901ed73e541daec...|[{1, person, 0.79...|2025-10-29 13:33:23|
|{6901ed73e541daec...|[{1, person, 0.74...|2025-10-29 13:33:23|
|{6901ed73e541daec...|[{1, person, 0.82...|2025-10-29 13:33:23|
|{6901ed73e541daec...|[{1, person, 0.82...|2025-10-29 13:33:23|
|{6901ed73e541daec...|[{1, person, 0.79...|2025-10-29 13:33:23|
|{6901ed73e541daec...|[{1, person, 0.81...|2025-10-29 13:33:23|
|{6901ed73e541daec...|[{1, person, 0.78...|2025-10-29 13:33:23|
|{6901ed73e541daec...|[{1, person, 0.78...|2025-10-29 13:33:23|
|{6901ed74e541daec...|[{1, person, 0.83...|2025-10-29 13:33:23|
|{6901ed74e541daec...|[

In [11]:
df.printSchema()

root
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)
 |-- detections: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- id: integer (nullable = true)
 |    |    |-- class: string (nullable = true)
 |    |    |-- confidence: double (nullable = true)
 |    |    |-- bbox: array (nullable = true)
 |    |    |    |-- element: double (containsNull = true)
 |-- timestamp: string (nullable = true)



In [20]:
from pyspark.sql.functions import explode , col

df_det = df.select(explode("detections").alias("det"))

df_det.show(truncate=False)


+----------------------------------------------------------------------------------------------------------------+
|det                                                                                                             |
+----------------------------------------------------------------------------------------------------------------+
|{1, person, 0.746698796749115, [216.02334594726562, 210.7078857421875, 436.31866455078125, 479.667236328125]}   |
|{1, person, 0.7781017422676086, [222.59445190429688, 214.45384216308594, 439.5824890136719, 479.5044250488281]} |
|{1, person, 0.7926472425460815, [226.48963928222656, 213.864501953125, 443.6326904296875, 479.43377685546875]}  |
|{1, person, 0.7923277616500854, [231.95619201660156, 212.75038146972656, 449.6343688964844, 479.5732116699219]} |
|{1, person, 0.7447474002838135, [238.20816040039062, 211.42039489746094, 456.18072509765625, 479.6798400878906]}|
|{1, person, 0.8264439702033997, [241.2360382080078, 210.433349609375, 458.99307

In [18]:
df_det = df.select(
    explode("detections").alias("det")
).select(
    "det.id",
    "det.class",
    "det.confidence",
)

df_det.show(truncate=False)


+---+------+------------------+
|id |class |confidence        |
+---+------+------------------+
|1  |person|0.746698796749115 |
|1  |person|0.7781017422676086|
|1  |person|0.7926472425460815|
|1  |person|0.7923277616500854|
|1  |person|0.7447474002838135|
|1  |person|0.8264439702033997|
|1  |person|0.8264439702033997|
|1  |person|0.7952414751052856|
|1  |person|0.8130254745483398|
|1  |person|0.7802901864051819|
|1  |person|0.7802901864051819|
|1  |person|0.836057722568512 |
|1  |person|0.8575061559677124|
|1  |person|0.8588272333145142|
|1  |person|0.8234952092170715|
|1  |person|0.8234952092170715|
|1  |person|0.787367045879364 |
|1  |person|0.849059522151947 |
|1  |dog   |0.7829587459564209|
|2  |dog   |0.7780360579490662|
+---+------+------------------+
only showing top 20 rows



In [23]:
count_person = df.select(
    explode("detections").alias("det")
).filter(
    col("det.class") == "person"
).count()

print("count of persons : ", count_person)


count of persons :  207


In [24]:
df_count_time = df.select(
    col("timestamp"),
    explode("detections").alias("det")
).filter(
    col("det.class") == "person"
).groupBy(
    "timestamp"
).count()

df_count_time.show(truncate=False)


+-------------------+-----+
|timestamp          |count|
+-------------------+-----+
|2025-10-29 13:35:16|15   |
|2025-10-29 13:35:07|16   |
|2025-10-29 13:33:24|6    |
|2025-10-29 13:33:22|1    |
|2025-10-29 13:35:09|12   |
|2025-10-29 13:35:05|15   |
|2025-10-29 13:35:03|15   |
|2025-10-29 13:33:23|11   |
|2025-10-29 13:35:11|15   |
|2025-10-29 13:35:04|16   |
|2025-10-29 13:35:15|14   |
|2025-10-29 13:35:12|3    |
|2025-10-29 13:35:08|17   |
|2025-10-29 13:35:14|17   |
|2025-10-29 13:35:02|2    |
|2025-10-29 13:35:13|3    |
|2025-10-29 13:35:06|16   |
|2025-10-29 13:35:10|13   |
+-------------------+-----+

